# Tutorial 2 Loading data from GAMMA into python

This tutorial covers the following topics:

1. loading dependencies
2. defining constants and other user input parameters
3. grouping data into multi-kernel objects (basically a list of single kernel objects that contain the offset tracking data for each window size or kernel)
4. basic visualisation



In [1]:
# loading dependencies

# this is a big list of dependencies that need to be loaded for the rest of the code to work.

import numpy as np # numerical calculation 
import pandas as pd # dataset manipulation
import numba # optimization with jit compiler
from numba import vectorize 

import glob # for file search
import copy # for deep copying data
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd # geographical dataset manipulation
import rasterio as rio # raster read/write
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon # shapefile/ geometry manipulation
from shapely.geometry.point import Point
import pyproj # geographical coordinate translation
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm # utm calculator

import matplotlib.pyplot as plt # visualization
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns # visualization
from matplotlib import rc_file_defaults # restore defaults at start of script from seaborn to matplotlib
rc_file_defaults()
# sns.set(style=None, color_codes=True)


import datetime # time and date parser

import configparser # read parameters from text file

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package

import SPOTSAR_main as sm


## user inputs

We need to define user inputs like file names, image width, plotting latitude and longitude limits

In [2]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# define map region of interest
lon_lims = [110.425, 110.45]
lat_lims = [-7.555, -7.535]

# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt2/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS2/"

# Set the regular expression pattern to match the file names
PATTERN1 = r"^c20200927_c20201113_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
PATTERN_CCS1 = r"^c20200927_c20201113_ccs_[0-9]+_[0-9]+$"

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

## the files first need to be sorted from small to largest window size.

In [3]:
#
# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)

# test if file ordering has worked
print(matching_files1)
print(matching_files_ccs1)


['c20200927_c20201113_disp_58_28.txt', 'c20200927_c20201113_disp_140_68.txt', 'c20200927_c20201113_disp_224_108.txt', 'c20200927_c20201113_disp_306_148.txt', 'c20200927_c20201113_disp_388_188.txt']
['c20200927_c20201113_ccs_58_28', 'c20200927_c20201113_ccs_140_68', 'c20200927_c20201113_ccs_224_108', 'c20200927_c20201113_ccs_306_148', 'c20200927_c20201113_ccs_388_188']


In [4]:

# load data from files into class multi-kernel (a collection of offset estimates of the same dates with different window sizes)
datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                            matching_files1,
                                            DIRECTORY_PATH_CCS,
                                            matching_files_ccs1,
                                            LAT_FILE,
                                            LON_FILE,
                                            HEADING,
                                            MEAN_INC,
                                            LINES_CCS,
                                            WIDTH_CCS)
# We need to assign some data not stored in the disp.txt files.
datastack.get_params_from_file_name()
datastack.get_latlon_from_file(WIDTH)
datastack.add_lat_lon_to_data(R_START,A_START)
datastack.crop_stack_ccs(R_STEP,A_STEP)
# the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
# Next we add all the offset data (disp.txt) to the stack
stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
# The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.



### basic visualisation

next we try some basic visualisation to see if it all worked. 

In [ ]:
%matplotlib osx
sm.plot.plot_vec_attr(datastack.Stack[2],'Phase',2,50,0.005,[0,360],5,SHADING,DEM_EXTENT)